In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import datetime as dt
from autogen import AssistantAgent
from inswitch.llm.model import get_openai_model_config
import time

app = Flask(__name__)
CORS(app)

llm_config = {"config_list": [get_openai_model_config()]}
assistant = AssistantAgent(name="assistant", llm_config=llm_config)

In [ ]:
intents = []
reports = []

def find_next_id_intent():
    """
        A method that retrieves the maximum ID plus one in order to add a new intent
    """
    if len(intents) == 0:
        return 1
    else :
        return max(intent.id for intent in intents) + 1

def get_intent_by_id(id_intent):
    """
        A method that retrieves an Intent by ID
    """
    intent = None
    for i in intents:
        if i.id == id_intent:
            intent = i
    
    return intent

def find_next_id_report():
    """
        A method that retrieves the maximum ID plus one in order to add a new report
    """
    if len(reports) == 0:
        return 1
    else :
        return max(report.id for report in reports) + 1


In [ ]:
class Intent:
    """
    A class used to represent an Intent

    Attributes
    ----------
    id : int
        a unique id
    date : date
        the creation date of the intent
    lastUpdateDate : date
        the date of the last update of the intent
    author : str
        the author of the intent
    content : str
        the content of the intent

    Methods
    -------
    to_dict(self)
        Returns the intent as a dictionary
    
    getId(self)
        Returns the id of the intent
    """
    def __init__(self, author, content):
        self.id = find_next_id_intent()
        self.date = dt.datetime.now()
        self.lastUpdateDate = dt.datetime.now()
        self.author = author
        self.content = content
    
    def to_dict(self):
        return {
            "id": self.id,
            "date": self.date,
            "lastUpdateDate": self.lastUpdateDate,
            "author": self.author,
            "content": self.content
        }
    
    def getId(self):
        return self.id

# Adding some intents to the list
intents.append(Intent("Person 1", "Content of the intent n°1"))
intents.append(Intent("Person 2", "Content of the intent n°2"))
intents.append(Intent("Person 3", "Content of the intent n°3"))

In [ ]:
class Report:
    """
    A class used to represent an Intent Report

    Attributes
    ----------
    id : int
        a unique id linked with the intent
    idIntent : int
        id of the intent
    agentName : str
        the name of the agent who handles the intent
    date : date
        the date the intent was handled
    response : str
        the response of the handle
    executionTime : float
        the execution time of the handle

    Methods
    -------
    to_dict(self)
        Returns the intent report as a dictionary
    """

    def __init__(self, idIntent, agentName, response, executionTime):
        self.id = find_next_id_report()
        self.idIntent = idIntent
        self.agentName = agentName
        self.date = dt.datetime.now()
        self.response = response
        self.executionTime = executionTime
    
    def to_dict(self):
        return {
            "id": self.id,
            "idIntent": self.idIntent,
            "agentName": self.agentName,
            "date": self.date,
            "response": self.response,
            "executionTime": self.executionTime
        }


In [ ]:
# GET METHODS

# Retrieves the list of intent objects

@app.get("/intent")
def get_intents():
    if(len(intents) == 0):
        return '', 204
    return jsonify([intent.to_dict() for intent in intents]), 200


# Retrieves the list of intent and return it as RDF in JSON-LD format

@app.get("/intent/json-ld")
def get_intents_json_ld():
    if(len(intents) == 0):
        return '', 204
    
    # Definition of the context
    jsonld_data = {
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema# "
        }
    }
    jsonld_data["intents"] = []

    for intent in intents:
        intent_data = {
            "@id": f"ex:{intent.id}",
            "@type": "icm:Intent",
            "ex:author": {
                "@id": f"ex:person/{intent.author.replace(' ', '_')}",
                "@type": "foaf:Person",
                "foaf:name": intent.author
            },
            "ex:content": intent.content,
            "dcterms:date": {
                "@value": intent.date.isoformat(),
                "@type": "xsd:dateTime"
            },
            "dcterms:lastUpdateDate": {
                "@value": intent.lastUpdateDate.isoformat(),
                "@type": "xsd:dateTime"
            }
        }
        jsonld_data["intents"].append(intent_data)

    return jsonify(jsonld_data), 200


# Retrieves an Intent by ID

@app.get("/intent/<int:id_intent>")
def get_intent_from_id_json(id_intent):
    intent = get_intent_by_id(id_intent)

    # If intent exists
    if intent :
        return intent.to_dict(), 200
    else :
        return {"error": "No intent with id " + str(id_intent)}, 404


# Retrieves an Intent by ID and return it as RDF in JSON-LD format

@app.get("/intent/<int:id_intent>/json-ld")
def get_intent_jsonld_by_id(id_intent):
    intent = get_intent_by_id(id_intent)

    # If intent exists
    if intent:
        jsonld_data = {
            # Definition of the context
            "@context": {
                "ex": "http://example.com/intent/",
                "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
                "foaf": "http://xmlns.com/foaf/0.1/",
                "dcterms": "http://purl.org/dc/terms/",
                "xsd": "http://www.w3.org/2001/XMLSchema# "
            },
            "@id": f"ex:{intent.id}",
            "@type": "icm:Intent",
            "ex:author": {
                "@id": f"ex:person/{intent.author.replace(' ', '_')}",
                "@type": "foaf:Person",
                "foaf:name": intent.author
            },
            "ex:content": intent.content,
            "dcterms:date": {
                "@value": intent.date.isoformat(),
                "@type": "xsd:dateTime"
            },
            "dcterms:lastUpdateDate": {
                "@value": intent.lastUpdateDate.isoformat(),
                "@type": "xsd:dateTime"
            }
        }

        return jsonld_data, 200
    else :
        return {"error": "No intent with id " + str(id_intent)}, 404
    


# Intent Report

# Retrieves the list of IntentReport of an Intent

@app.get("/intent/<int:id_intent>/intentReport")
def get_intent_report_of_intent_id(id_intent):
    intent = get_intent_by_id(id_intent)

    # If intent exists
    if intent :
        return [report.to_dict() for report in reports if report.idIntent == intent.getId()], 200
    else :
        return {"error": "No intent with id " + str(id_intent)}, 404

# Retrieves an IntentReport of an Intent

@app.get("/intent/<int:id_intent>/intentReport/<int:id_report>")
def get_intent_report_from_id_of_intent_id(id_intent, id_report):
    intent = get_intent_by_id(id_intent)

    # If intent exists
    if intent :
        report = [report.to_dict() for report in reports if report.idIntent == intent.getId() and report.id == id_report]

        # If report of the intent exists
        if report:
            return report[0], 200
        else :
            return {"error": "No report with id " + str(id_report) + " of intent with id " + str(id_intent)}, 404
    else :
        return {"error": "No intent with id " + str(id_intent)}, 404
    

@app.get("/json-ld")
def get_json_ld():
    if(len(intents) == 0):
        return '', 204
    
    # Definition of the context
    jsonld_data = {
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema# "
        }
    }
    jsonld_data["@graph"] = []

    intents_list = []

    for intent in intents:
        intent_data = {
            "@id": f"ex:{intent.id}",
            "@type": "icm:Intent",
            "ex:author": {
                "@id": f"ex:person/{intent.author.replace(' ', '_')}",
                "@type": "foaf:Person",
                "foaf:name": intent.author
            },
            "ex:content": intent.content,
            "dcterms:date": {
                "@value": intent.date.isoformat(),
                "@type": "xsd:dateTime"
            },
            "dcterms:lastUpdateDate": {
                "@value": intent.lastUpdateDate.isoformat(),
                "@type": "xsd:dateTime"
            }
        }
        intents_list.append(intent_data)
    
    jsonld_data["@graph"].append(intents_list)
    
    if len(reports) > 0:
        reports_list = []

        for report in reports:
            report_data = {
                "@id": f"ex:{report.idIntent}/intentReport/{report.id}",
                "@type": "icm:IntentReport",
                "ex:intent": {
                    "@id": f"ex:{report.idIntent}"
                },
                "ex:agentName": report.agentName,
                "dcterms:date": {
                    "@type": "xsd:dateTime",
                    "@value": report.date
                },
                "ex:response": report.response,
                "ex:executionTime": {
                    "@type": "xsd:float",
                    "@value": report.executionTime
                }
            }
            reports_list.append(report_data)
        
        jsonld_data["@graph"].append(reports_list)

    return jsonify(jsonld_data), 200

In [ ]:
# POST METHODS

# Creates an Intent

@app.post("/intent")
def add_intent():
    if request.is_json:
        """
        intent = request.get_json()
        intent.id = _find_next_id_intent()
        intent.date = dt.datetime.now()
        intents.append(intent)
        """
        added_intent = request.get_json()
        intent = Intent(added_intent["author"], added_intent["content"])
        intents.append(intent)

        # Handle of the intent

        startTime = time.time()

        response = assistant.generate_reply(
            messages=[{"role": "user", "content": added_intent["content"]}]
        )

        # Cleaning of the response

        if(response.endswith("TERMINATE")):
            response = response.replace("TERMINATE", "").strip()
        
        report = Report(intent.getId(), assistant.name, response, (time.time() - startTime))

        reports.append(report)
        
        return {"intent":intent.to_dict(), "report":report.to_dict()}, 201
    return {"error": "Request must be JSON"}, 415

In [ ]:
# PATCH METHODS

# Partially updates an Intent

@app.patch("/intent/<int:id_intent>")
def update_intent(id_intent):
    intent = get_intent_by_id(id_intent)

    # If intent exists
    if intent :
        updated_intent = request.get_json()

        if "content" in updated_intent and updated_intent["content"] != "":
            intent.content = updated_intent["content"] 
            intent.lastUpdateDate = dt.datetime.now()

        return intent.to_dict(), 200

    else:
        return {"error": "No intent with id " + str(id_intent)}, 404

In [ ]:
# DELETE METHODS

# Deletes an Intent

@app.delete("/intent/<int:id_intent>")
def delete_intent(id_intent):
    intent = get_intent_by_id(id_intent)

    # If intent exists
    if intent :
        index = intents.index(intent)
        intents.pop(index)

        return {"message": "Intent with id " + str(id_intent) + " deleted"}, 200
    else :
        return {"error": "No intent with id " + str(id_intent)}, 404


# Deletes an IntentReport of an Intent

@app.delete("/intent/<int:id_intent>/intentReport/<int:id_report>")
def delete_intent_report_from_id_of_intent_id(id_intent, id_report):
    intent = get_intent_by_id(id_intent)

    # If intent exists
    if intent :
        report = [report for report in reports if report.idIntent == intent.getId() and report.id == id_report]

        # If report of the intent exists
        if report:
            index = reports.index(report[0])
            reports.pop(index)

            return {"message": "IntentReport with id " + str(id_report) + " of intent with id " + str(id_report) + " deleted"}, 200
        else :
            return {"error": "No report with id " + str(id_report) + " of intent with id " + str(id_intent)}, 404
    else :
        return {"error": "No intent with id " + str(id_intent)}, 404

In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)